In [1]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Format Data

In [2]:
header = ['letter', 'x-box', 'y-box', 'width', 'height', 'totpix', 
         'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar',
         'x-edge', 'x-edge-y', 'y-edge', 'y-edge-x']
letters = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data', names=header)
# letters = letters.set_index('letter') 
letters = letters.sort_values('letter') # order rows alphabetically
letters = letters.reset_index(drop=True)
letters.head()

,letter,x-box,y-box,width,height,totpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybar,xy2bar,x-edge,x-edge-y,y-edge,y-edge-x
0,A,4,9,6,6,2,9,5,3,1,8,1,8,2,7,2,8
1,A,3,6,5,4,1,7,5,3,1,6,1,8,2,7,2,7
2,A,5,9,6,7,7,8,8,8,4,6,6,8,3,8,8,4
3,A,4,10,7,7,5,7,5,2,3,5,2,6,3,7,4,4
4,A,4,9,7,7,5,8,5,2,4,6,1,5,3,5,4,5


In [3]:
# make Y 
ones_pos = [1]*9940 # 9940 is the number of datapoints with letters A-M
ones_neg = [-1]*(20000-9940) # the number of the rest of the datapoints with letters N-Z
Y = np.array(ones_pos + ones_neg)
print(Y.shape)
# make X_and_Y
letters_withY = letters
letters_withY['Y'] = Y
# letters_withY
X_and_Y = letters_withY.drop('letter',1)
X_and_Y.head()

(20000,)


,x-box,y-box,width,height,totpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybar,xy2bar,x-edge,x-edge-y,y-edge,y-edge-x,Y
0,4,9,6,6,2,9,5,3,1,8,1,8,2,7,2,8,1
1,3,6,5,4,1,7,5,3,1,6,1,8,2,7,2,7,1
2,5,9,6,7,7,8,8,8,4,6,6,8,3,8,8,4,1
3,4,10,7,7,5,7,5,2,3,5,2,6,3,7,4,4,1
4,4,9,7,7,5,8,5,2,4,6,1,5,3,5,4,5,1


### Create 5 Trials - Training and Testing Sets

In [4]:
from sklearn.model_selection import train_test_split

XY_train1, XY_test1 = train_test_split(X_and_Y, test_size=15000, shuffle=True)
XY_train2, XY_test2 = train_test_split(X_and_Y, test_size=15000, shuffle=True)
XY_train3, XY_test3 = train_test_split(X_and_Y, test_size=15000, shuffle=True)
XY_train4, XY_test4 = train_test_split(X_and_Y, test_size=15000, shuffle=True)
XY_train5, XY_test5 = train_test_split(X_and_Y, test_size=15000, shuffle=True)

In [5]:
print(XY_train1.shape)
print(XY_test1.shape)
XY_train1.head()

(5000, 17)
(15000, 17)


,x-box,y-box,width,height,totpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybar,xy2bar,x-edge,x-edge-y,y-edge,y-edge-x,Y
4687,3,9,5,6,4,7,6,7,5,6,6,8,2,7,5,11,1
3055,6,10,6,5,3,8,6,5,6,12,4,7,5,7,5,9,1
17615,5,6,5,4,4,2,11,2,3,10,10,8,5,11,1,7,-1
18047,3,9,4,7,3,8,7,4,4,7,6,9,2,8,4,8,-1
18231,3,5,5,4,2,10,7,1,8,11,3,7,2,8,3,9,-1


In [6]:
%%time
#import warnings
# there are a lot of convergence warnings for some params, however be careful with this!!
# sometimes you need to see those wanrings, and now we've screwed tha tup for the whole notebook from here on!!
#warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression 
import numpy as np
from sklearn.model_selection import GridSearchCV

# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-9, 9, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-9, 9, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']}
                ]

trialnum = 0
accuracy_sum = 0 # sum of top accuracy to later calculate the average of all 5 trials
roc_sum = 0 # sum of top roc score to later calculate the average of all 5 trials
f1_sum = 0 # sum of top accuracy to later calculate the average of all 5 trials
accuracy_scores = []
roc_scores = []
f1_scores = []
all_accuracy_models = []
all_roc_models = []
all_f1_models = []

# for every trial
for trial in [XY_train1, XY_train2, XY_train3, XY_train4, XY_train5]:
 
    trialnum = trialnum + 1
    X_l = trial.drop(['Y'],1)
    y_l = trial['Y']
    
    # Create grid search 
    clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                       scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                       verbose=0)

    # Fit grid search
    best_model = clf.fit(X_l, y_l)
    
    print("------------------------------------------------------------------------------------")
    print("RESULTS FOR TRIAL:")
    print(trialnum)
    print("------------------------------------------------------------------------------------")
    
    # the detailed results of the whole model selection search...
#     print(best_model.cv_results_)

    print("---------------BEST MODEL FOR ACCURACY: ----------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ] )
    print("---WITH ACCURACY: ---")
    current_accuracy = best_model.cv_results_['mean_test_accuracy'][ np.argmax(best_model.cv_results_['mean_test_accuracy']) ]
    print(current_accuracy)
    accuracy_sum = accuracy_sum + current_accuracy
    accuracy_scores.append(current_accuracy)
    all_accuracy_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ] )
    
    print("---------------BEST MODEL FOR ROC: ---------------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc_ovr']) ] )
    print("---WITH ROC: ---")
    current_roc = best_model.cv_results_['mean_test_roc_auc_ovr'][ np.argmax(best_model.cv_results_['mean_test_roc_auc_ovr']) ]
    print(current_roc)
    roc_sum = roc_sum + current_roc
    roc_scores.append(current_roc)
    all_roc_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc_ovr']) ] )

    
    print("---------------BEST MODEL FOR F1: ----------------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro']) ] )
    print("---WITH F1: ---")    
    current_f1 = best_model.cv_results_['mean_test_f1_micro'][ np.argmax(best_model.cv_results_['mean_test_f1_micro']) ]
    print(current_f1)
    f1_sum = f1_sum + current_f1
    f1_scores.append(current_f1)
    all_f1_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro']) ] )

    
    # below (optional): check that the above outputs actually show best scores

#     print results just to check alignment with the above
#     results = pd.DataFrame( best_model.cv_results_['params'] ) # parameter settings for best model
#     grab the accuracy score resulting from those parameters
#     results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
#     results['score_roc'] = best_model.cv_results_['mean_test_roc_auc_ovr']
#     results['score_f1'] = best_model.cv_results_['mean_test_f1_micro']
#     get rid of classifier__XX in columns
#     cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
#     results.columns = cols
#     print(results)


------------------------------------------------------------------------------------
RESULTS FOR TRIAL:
1
------------------------------------------------------------------------------------
---------------BEST MODEL FOR ACCURACY: ----------
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 177.82794100389228, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
---WITH ACCURACY: ---
0.7213999999999999
---------------BEST MODEL FOR ROC: ---------------
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 31622.776601683792, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
---WITH ROC: ---
0.8099302224184749
---------------BEST MODEL FOR F1: ----------------
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 177.82794100389228, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
---WITH F1: ---
0.7213999999999999
------------------------------------------------------------------------------------
RESULTS FOR TRIAL:

### Get Train Metrics

In [7]:
print("===== ACCURACY SCORES: =====")
print(accuracy_scores)
print("===== ROC SCORES: =====")
print(roc_scores)
print("===== F1 SCORES: =====")
print(f1_scores)

===== ACCURACY SCORES: =====
[0.7213999999999999, 0.7251999999999998, 0.7178, 0.733, 0.735]
===== ROC SCORES: =====
[0.8099302224184749, 0.8057376744326238, 0.8062200765400075, 0.8217347737806951, 0.8234107724782899]
===== F1 SCORES: =====
[0.7213999999999999, 0.7251999999999998, 0.7178, 0.733, 0.735]


In [8]:
print("================ BEST ACCURACY MODEL IN TRAINING: ==================")
print(all_accuracy_models[ np.argmax(accuracy_scores) ])
print("WITH ACCURACY:")
print(max(accuracy_scores))
print("================ BEST ROC MODEL IN TRAINING: ==================")
print(all_roc_models[ np.argmax(roc_scores) ])
print("WITH ROC SCORE:")
print(max(roc_scores))
print("================ BEST F1 MODEL IN TRAINING: ==================")
print(all_f1_models[ np.argmax(f1_scores) ])
print("WITH F1 SCORE:")
print(max(f1_scores))

================ BEST ACCURACY MODEL IN TRAINING: ==================
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 177.82794100389228, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
WITH ACCURACY:
0.735
================ BEST ROC MODEL IN TRAINING: ==================
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 1000000000.0, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
WITH ROC SCORE:
0.8234107724782899
================ BEST F1 MODEL IN TRAINING: ==================
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 177.82794100389228, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
WITH F1 SCORE:
0.735


In [9]:
print("================ AVERAGE ACCURACY ON TRAIN SET: ==================")
print(accuracy_sum / 5)
print("================ AVERAGE ROC SCORE ON TRAIN SET: ==================")
print(roc_sum / 5)
print("================ AVERAGE F1 SCORE ON TRAIN SET: ==================")
print(f1_sum / 5)


================ AVERAGE ACCURACY ON TRAIN SET: ==================
0.7264799999999999
================ AVERAGE ROC SCORE ON TRAIN SET: ==================
0.8134067039300182
================ AVERAGE F1 SCORE ON TRAIN SET: ==================
0.7264799999999999


### Get Test Metrics

In [10]:
performance_AccModel = pd.DataFrame(index=['acc', 'roc', 'f1'], columns=['Trial 1', 'Trial 2', 'Trial 3', 'Trial 4', 'Trial 5'])
performance_AccModel

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5
acc,NaN,NaN,NaN,NaN,NaN
roc,NaN,NaN,NaN,NaN,NaN
f1,NaN,NaN,NaN,NaN,NaN


#### Best Training Accuracy & F1 Score Model Results after Testing

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

# test performance of model rated as best for accuracy and f1 score on training set

# test performance on trial 1 test set
clf = LogisticRegression(penalty='l1', C=1.0, solver='saga', max_iter=10000).fit(XY_train1.drop(['Y'],1), XY_train1['Y'])
pred = clf.predict(XY_test1.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 1'] = accuracy_score(XY_test1['Y'], pred)
performance_AccModel.loc['roc', 'Trial 1'] = roc_auc_score(XY_test1['Y'], pred)
performance_AccModel.loc['f1', 'Trial 1'] = f1_score(XY_test1['Y'], pred)

# test performance on trial 2 test set
clf = LogisticRegression(penalty='l1', C=1.0, solver='saga', max_iter=10000).fit(XY_train2.drop(['Y'],1), XY_train2['Y'])
pred = clf.predict(XY_test2.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 2'] = accuracy_score(XY_test2['Y'], pred)
performance_AccModel.loc['roc', 'Trial 2'] = roc_auc_score(XY_test2['Y'], pred)
performance_AccModel.loc['f1', 'Trial 2'] = f1_score(XY_test2['Y'], pred)

# test performance on trial 3 test set
clf = LogisticRegression(penalty='l1', C=1.0, solver='saga', max_iter=10000).fit(XY_train3.drop(['Y'],1), XY_train3['Y'])
pred = clf.predict(XY_test3.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 3'] = accuracy_score(XY_test3['Y'], pred)
performance_AccModel.loc['roc', 'Trial 3'] = roc_auc_score(XY_test3['Y'], pred)
performance_AccModel.loc['f1', 'Trial 3'] = f1_score(XY_test3['Y'], pred)

# test performance on trial 4 test set
clf = LogisticRegression(penalty='l1', C=1.0, solver='saga', max_iter=10000).fit(XY_train4.drop(['Y'],1), XY_train4['Y'])
pred = clf.predict(XY_test4.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 4'] = accuracy_score(XY_test4['Y'], pred)
performance_AccModel.loc['roc', 'Trial 4'] = roc_auc_score(XY_test4['Y'], pred)
performance_AccModel.loc['f1', 'Trial 4'] = f1_score(XY_test4['Y'], pred)

# test performance on trial 5 test set
clf = LogisticRegression(penalty='l1', C=1.0, solver='saga', max_iter=10000).fit(XY_train5.drop(['Y'],1), XY_train5['Y'])
pred = clf.predict(XY_test5.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 5'] = accuracy_score(XY_test5['Y'], pred)
performance_AccModel.loc['roc', 'Trial 5'] = roc_auc_score(XY_test5['Y'], pred)
performance_AccModel.loc['f1', 'Trial 5'] = f1_score(XY_test5['Y'], pred)

performance_AccModel

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5
acc,0.728267,0.725533,0.7268,0.725733,0.7198
roc,0.728314,0.725669,0.726818,0.725864,0.719897
f1,0.730566,0.727044,0.72337,0.728305,0.721821


#### Best Training ROC Score Model Results after Testing

In [12]:
# test performance of model rated as best for roc on training set
performance_ROCModel = pd.DataFrame(index=['acc', 'roc', 'f1'], columns=['Trial 1', 'Trial 2', 'Trial 3', 'Trial 4', 'Trial 5'])

# test performance on trial 1 test set
clf = LogisticRegression(penalty='l2', C=177.82794100389228, solver='lbfgs', max_iter=10000).fit(XY_train1.drop(['Y'],1), XY_train1['Y'])
pred = clf.predict(XY_test1.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 1'] = accuracy_score(XY_test1['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 1'] = roc_auc_score(XY_test1['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 1'] = f1_score(XY_test1['Y'], pred)

# test performance on trial 2 test set
clf = LogisticRegression(penalty='l2', C=177.82794100389228, solver='lbfgs', max_iter=10000).fit(XY_train2.drop(['Y'],1), XY_train2['Y'])
pred = clf.predict(XY_test2.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 2'] = accuracy_score(XY_test2['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 2'] = roc_auc_score(XY_test2['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 2'] = f1_score(XY_test2['Y'], pred)

# test performance on trial 3 test set
clf = LogisticRegression(penalty='l2', C=177.82794100389228, solver='lbfgs', max_iter=10000).fit(XY_train3.drop(['Y'],1), XY_train3['Y'])
pred = clf.predict(XY_test3.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 3'] = accuracy_score(XY_test3['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 3'] = roc_auc_score(XY_test3['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 3'] = f1_score(XY_test3['Y'], pred)

# test performance on trial 4 test set
clf = LogisticRegression(penalty='l2', C=177.82794100389228, solver='lbfgs', max_iter=10000).fit(XY_train4.drop(['Y'],1), XY_train4['Y'])
pred = clf.predict(XY_test4.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 4'] = accuracy_score(XY_test4['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 4'] = roc_auc_score(XY_test4['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 4'] = f1_score(XY_test4['Y'], pred)

# test performance on trial 5 test set
clf = LogisticRegression(penalty='l2', C=177.82794100389228, solver='lbfgs', max_iter=10000).fit(XY_train5.drop(['Y'],1), XY_train5['Y'])
pred = clf.predict(XY_test5.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 5'] = accuracy_score(XY_test5['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 5'] = roc_auc_score(XY_test5['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 5'] = f1_score(XY_test5['Y'], pred)

performance_ROCModel

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5
acc,0.728,0.724667,0.726733,0.725067,0.720133
roc,0.728048,0.724807,0.726749,0.725196,0.720232
f1,0.730337,0.726309,0.723657,0.727609,0.722208


## Get Final Logistic Regression Results

In [14]:
print("================ BEST ACCURACY MODEL IN LOGISTIC REGRESSION: ==================")
print(all_accuracy_models[ np.argmax(np.array(performance_AccModel.iloc[0])) ])
print("WITH ACCURACY:")
print((np.array(performance_AccModel.iloc[0])).mean())
print("================ BEST ROC MODEL IN LOGISTIC REGRESSION: ==================")
print(all_roc_models[ np.argmax(np.array(performance_ROCModel.iloc[1])) ])
print("WITH ROC SCORE:")
print((performance_ROCModel.iloc[1]).mean())
print("================ BEST F1 MODEL IN LOGISTIC REGRESSION: ==================")
print(all_f1_models[ np.argmax(np.array(performance_AccModel.iloc[2])) ])
print("WITH F1 SCORE:")
print((np.array(performance_AccModel.iloc[2])).mean())

================ BEST ACCURACY MODEL IN LOGISTIC REGRESSION: ==================
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 177.82794100389228, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
WITH ACCURACY:
0.7252266666666667
================ BEST ROC MODEL IN LOGISTIC REGRESSION: ==================
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 31622.776601683792, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
WITH ROC SCORE:
0.7250066218471684
================ BEST F1 MODEL IN LOGISTIC REGRESSION: ==================
{'classifier': LogisticRegression(max_iter=10000), 'classifier__C': 177.82794100389228, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
WITH F1 SCORE:
0.7262212243607072
